In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt

In [3]:
task_datasets = []

# Task 1
task1_data = np.load('./cwru1_X_train.npy')
task1_labels = np.load('./cwru2_y_train.npy')

# Task 2
task2_data = np.load('./cwru2_X_train.npy')
task2_labels = np.load('./cwru2_y_train.npy')

# Task 3
task3_data = np.load('./cwru3_X_train.npy')
task3_labels = np.load('./cwru3_y_train.npy')

# Task 4
task4_data = np.load('./cwru4_X_train.npy')
task4_labels = np.load('./cwru4_y_train.npy')

In [4]:
# Convert the data to PyTorch tensors
# Task 1
task1_data = torch.from_numpy(task1_data)
task1_labels = torch.from_numpy(task1_labels)

# Task 2
task2_data = torch.from_numpy(task2_data)
task2_labels = torch.from_numpy(task2_labels)

# Task 3
task3_data = torch.from_numpy(task3_data)
task3_labels = torch.from_numpy(task3_labels)

# Task 4
task4_data = torch.from_numpy(task4_data).double()
task4_labels = torch.from_numpy(task4_labels).double()

In [5]:
# data transpose
task1_data = task1_data.transpose(1, 2)
task2_data = task2_data.transpose(1, 2)
task3_data = task3_data.transpose(1, 2)
task4_data = task4_data.transpose(1, 2)

In [6]:
# dataset for task 1~4
dataset1 = torch.utils.data.TensorDataset(task1_data, task1_labels)
dataset2 = torch.utils.data.TensorDataset(task2_data, task2_labels)
dataset3 = torch.utils.data.TensorDataset(task3_data, task3_labels)
dataset4 = torch.utils.data.TensorDataset(task4_data, task4_labels)

In [7]:
# 划分训练集和测试集
train_size = int(0.8 * len(dataset1))
test_size = len(dataset1) - train_size

train_dataset1, test_dataset1 = random_split(dataset1, [train_size, test_size])
train_dataset2, test_dataset2 = random_split(dataset2, [train_size, test_size])
train_dataset3, test_dataset3 = random_split(dataset3, [train_size, test_size])
train_dataset4, test_dataset4 = random_split(dataset4, [train_size, test_size])

In [8]:
task_datasets.append((train_dataset1, test_dataset1))
task_datasets.append((train_dataset2, test_dataset2))
task_datasets.append((train_dataset3, test_dataset3))
task_datasets.append((train_dataset4, test_dataset4))

In [9]:
# 定义WDCNN模型
class WDCNN(nn.Module):
    def __init__(self, num_classes):
        super(WDCNN, self).__init__()
        self.conv1 = nn.Conv1d(2, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 256, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [11]:
!pip install python-csv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f2/94/3af39d34be01a24a6e65433d19e107099374224905f1e0cc6bbe1fd22a2f/argparse-1.4.0-py2.py3-none-any.whl (23 kB)
     |████████████████████████████████| 96 kB 699 kB/s eta 0:00:011
     |████████████████████████████████| 131 kB 1.6 MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /home/ic-lab/miniconda3/envs/torch_v1.9_gpu/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-394aou_d/demjson_72bac856d4964ad4a98f3d631dc36e44/setup.py'"'"'; __file__='"'"'/tmp/pip-install-394aou_d/demjson_72bac856d4964ad4a98f3d631dc36e44/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --eg

In [12]:
import csv

In [20]:
# 创建WDCNN模型实例
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model and optimizers
model = WDCNN(num_classes=10).to(device)
model.double()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# training and testing
num_epochs = 5
task_accuracies = []



# for i in range(4): # each task
for i in range(4): # each task
    print(f"Training Task {i+1}")
    
    # Reset the parameter of model
    model.apply(lambda m: setattr(m, 'prev_weights', None))
    
    train_loader = torch.utils.data.DataLoader(task_datasets[i][0], batch_size=32, shuffle=True)
    
    # Training
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs = inputs.to(device, dtype=torch.double)
            labels = labels.to(device, dtype=torch.long)
            
            # inputs = inputs.to(device)  # 移动到设备上
            # labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs ,labels)
            loss.backward()
            optimizer.step()
    
    model.eval()
    accuracies = []
    for j in range(4):
        confidence_scores = []
        # test_subset_indices = range(j * 2000, (j+1) * 2000)
        # test_subset = torch.utils.data.Subset(test_dataset, test_subset_indices)
        test_loader = torch.utils.data.DataLoader(task_datasets[j][1], batch_size=32, shuffle=False)
        total = 0
        correct = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device, dtype=torch.double)
            labels = labels.to(device, dtype=torch.long)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            probabilities = torch.softmax(outputs, dim=1)
            confidence_scores.extend(probabilities.tolist())
        accuracy = 100 * correct / total
        accuracies.append(accuracy)
        
        # confidence_scores = np.array(confidence_scores)
        # confidence_scores /= np.sum(confidence_scores, axis=1, keepdims=True)
        save_path = f'train_on_task_{i+1}_confidence_scores_on_task_{j+1}.csv'
        with open(save_path, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(confidence_scores)
        
    task_accuracies.append(accuracies)

Training Task 1
Training Task 2
Training Task 3
Training Task 4
